# --- Part Two ---
You quickly reach the farthest point of the loop, but the animal never emerges. Maybe its nest is within the area enclosed by the loop?

To determine whether it's even worth taking the time to search for such a nest, you should calculate how many tiles are contained within the loop. For example:
```
...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
...........
```

The above loop encloses merely four tiles - the two pairs of . in the southwest and southeast (marked I below). The middle . tiles (marked O below) are not in the loop. Here is the same loop again with those regions marked:
```
...........
.S-------7.
.|F-----7|.
.||OOOOO||.
.||OOOOO||.
.|L-7OF-J|.
.|II|O|II|.
.L--JOL--J.
.....O.....
```
In fact, there doesn't even need to be a full tile path to the outside for tiles to count as outside the loop - squeezing between pipes is also allowed! Here, I is still within the loop and O is still outside the loop:
```
..........
.S------7.
.|F----7|.
.||OOOO||.
.||OOOO||.
.|L-7F-J|.
.|II||II|.
.L--JL--J.
..........
```
In both of the above examples, 4 tiles are enclosed by the loop.

Here's a larger example:
```
.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...
```
The above sketch has many random bits of ground, some of which are in the loop (I) and some of which are outside it (O):
```
OF----7F7F7F7F-7OOOO
O|F--7||||||||FJOOOO
O||OFJ||||||||L7OOOO
FJL7L7LJLJ||LJIL-7OO
L--JOL7IIILJS7F-7L7O
OOOOF-JIIF7FJ|L7L7L7
OOOOL7IF7||L7|IL7L7|
OOOOO|FJLJ|FJ|F7|OLJ
OOOOFJL-7O||O||||OOO
OOOOL---JOLJOLJLJOOO
```
In this larger example, 8 tiles are enclosed by the loop.

Any tile that isn't part of the main loop can count as being enclosed by the loop. Here's another example with many bits of junk pipe lying around that aren't connected to the main loop at all:
```
FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
```
Here are just the tiles that are enclosed by the loop marked with I:
```
FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJIF7FJ-
L---JF-JLJIIIIFJLJJ7
|F|F-JF---7IIIL7L|7|
|FFJF7L7F-JF7IIL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
```
In this last example, 10 tiles are enclosed by the loop.

Figure out whether you have time to search for the nest by calculating the area within the loop. **How many tiles are enclosed by the loop?**

In [1]:
from utilities import get_lines
from collections import namedtuple
import pandas as pd

In [2]:
lines = get_lines('sample')

In [3]:
def get_S_location(lines):
    for i, line in enumerate(lines):
        if 'S' in line:
            row = i
    col = lines[row].index('S')
    return row,col

In [4]:
pipes = {'|':'NS',
         '-':'EW',
         'L':'NE',
         'J':'NW',
         '7':'SW',
         'F':'SE',
         'S':'start'
        }

In [5]:
max_row = len(lines)-1
max_col = len(lines[0])-1
max_row, max_col

(8, 10)

In [6]:
def is_valid_loc(i,j):
    if (0<=i<=max_row)&(0<=j<=max_col):
        return True
    return False

In [7]:
class Tile:
    def __init__(self, row, col, pipe):
        self.row = row
        self.col = col
        self.pipe = pipe
        self.N = False
        self.E = False
        self.S = False
        self.W = False
        self.is_start = False
        self.step = 0
        self.from_dir = None
        self.is_path = False
    
    Loc = namedtuple('Loc','row col pipe')
    
    def __repr__(self):
        return f'({self.row}, {self.col}, {self.pipe})'
    
    
    def check_NS(self): #  |
        if self.pipe=='|':
            if is_valid_loc(self.row-1,self.col):
                self.N = True
                return True
            if is_valid_loc(self.row+1,self.col):
                self.S = True
                return True
        return False
    
    def check_EW(self): #  -
        if self.pipe=='-':
            if is_valid_loc(self.row,self.col+1):
                self.E = True
                return True
            if is_valid_loc(self.row,self.col-1):
                self.W = True
                return True
        return False
    
    def check_NE(self): #  L
        if self.pipe=='L':
            if is_valid_loc(self.row-1,self.col):
                self.N = True
                return True
            if is_valid_loc(self.row,self.col+1):
                self.E = True
                return True
        return False
    
    def check_NW(self): #  J
        if self.pipe=='J':
            if is_valid_loc(self.row-1,self.col):
                self.N = True
                return True
            if is_valid_loc(self.row,self.col-1):
                self.W = True
                return True
        return False
    
    def check_SW(self): #  7
        if self.pipe=='7':
            if is_valid_loc(self.row+1,self.col):
                self.S = True
                return True
            if is_valid_loc(self.row,self.col-1):
                self.W = True
                return True
        return False
    
    def check_SE(self): #  F
        if self.pipe=='F':
            if is_valid_loc(self.row+1,self.col):
                self.S = True
                return True
            if is_valid_loc(self.row,self.col+1):
                self.E = True
                return True
        return False
    
    def set_start(self): #  S
        if self.pipe=='S':
            self.is_start = True
            return 1
        return 0
    
    def step_N(self):
        return (self.row-1,self.col)
    
    def step_E(self):
        return (self.row,self.col+1)
    
    def step_S(self):
        return (self.row+1,self.col)
    
    def step_W(self):
        return (self.row,self.col-1)

In [8]:
def find_first_step(start_tile):
    #  check the tile to the north, does it have a connection to the south?
    r, c = start_tile.step_N() 
    if is_valid_loc(r,c):
        next_tile = Tile(r,c,lines[r][c])
        if next_tile.check_NS()|next_tile.check_SE()|next_tile.check_SW():
            next_tile.from_dir = 'S'
            next_tile.is_path = True
            return next_tile
        
    #  check the tile to the east, does it have a connection to the west?
    r, c = start_tile.step_E()
    if is_valid_loc(r,c):
        next_tile = Tile(r,c,lines[r][c])
        if next_tile.check_EW()|next_tile.check_NW()|next_tile.check_SW():
            next_tile.from_dir = 'W'
            next_tile.is_path = True
            return next_tile
        
    #  check the tile to the south, does it have a connection to the north?
    r, c = start_tile.step_S()
    if is_valid_loc(r,c):
        next_tile = Tile(r,c,lines[r][c])
        if next_tile.check_NS()|next_tile.check_NW()|next_tile.check_NW():
            next_tile.from_dir = 'N'
            next_tile.is_path = True
            return next_tile
    
    #  no need to check to the west because one of the above HAS to return true
    return 'error'

In [9]:
def get_next_direction(from_dir,pipe_dirs):
    '''
    from_dir: char 
    pipe_dirs: str of two chars, one of which is the from_dir
    '''
    return str(set(pipe_dirs)-set(from_dir))[2]

In [10]:
def get_next_loc(curr_tile, next_direction):
    if next_direction=='N':
        r, c, = curr_tile.step_N()
    elif next_direction=='E':
        r, c = curr_tile.step_E()
    elif next_direction=='S':
        r, c = curr_tile.step_S()
    elif next_direction=='W':
        r, c = curr_tile.step_W()
    else:
        r, c = -1, -1
    return (r,c)

In [11]:
def get_came_from(next_direction):
    #  return the opposite on the compass rose
    compass = 'NESW'
    i = compass.index(next_direction)
    return compass[(i+2)%4]

In [12]:
def get_next_tile(curr_tile):
    # print('\n\ncame from direction ',curr_tile.from_dir)
    # don't need to check that direction
    # print('pipe ',curr_tile.pipe, 'directions ', pipes[curr_tile.pipe])
    next_direction = get_next_direction(curr_tile.from_dir, pipes[curr_tile.pipe])
    # print('\tnext direction ',next_direction)
    r, c = get_next_loc(curr_tile, next_direction)
    # print(r,c)
    next_tile = Tile(r,c,lines[r][c])
    next_tile.step = curr_tile.step+1
    next_tile.from_dir = get_came_from(next_direction)
    next_tile.is_path = True
    return next_tile

In [13]:
start_tile = Tile(*get_S_location(lines),'S')
start_tile.set_start()
print(f'start: {start_tile}')

curr_tile = find_first_step(start_tile)
curr_tile.step = 1
print(f'first tile: {curr_tile}')

path_tiles = [start_tile, curr_tile]

next_tile = get_next_tile(curr_tile)
print(f'next tile: {next_tile}')

steps = 0
while next_tile.pipe!='S':
# for i in range(5):
    path_tiles.append(next_tile)
    # print(curr_tile, curr_tile.step)
    curr_tile = next_tile
    next_tile = get_next_tile(curr_tile)
    steps+=1

print(f'returning to {next_tile} took {steps} steps')


start: (1, 1, S)
first tile: (1, 2, -)
next tile: (1, 3, -)
returning to (1, 1, S) took 44 steps


In [14]:
len(path_tiles)

46

In [15]:
path_tile_locs = [(tile.row, tile.col) for tile in path_tiles]
rows = list()
for row in range(max_row+1):
    row = list()
    for col in range(max_col+1):
        row.append('.')
    rows.append(row)

In [16]:
path_map = pd.DataFrame(rows)
for tile in path_tiles:
    path_map.at[tile.row,tile.col]=tile.pipe

In [17]:
# def count_inside_loop(row):
#     side_pipes= {'L','F','7','J','|','S'}
#     out_pipes = {'7','J','|','S'}
#     in_pipes = {'L','F','|','S'}
#     inside = False
#     inside_count = 0
#     last_pipe = '.'
#     for i, val in row.items():
#         # print(val)
#         if (val in in_pipes) and (inside==False) and (last_pipe not in {'7','J'}):
#             inside = True
#             print(f'\tinside at {i}, pipe {val}, count {inside_count}, last pipe {last_pipe}')
#             last_pipe=val
#         elif (val in out_pipes) and (inside==True) and (last_pipe not in {'L','F'}):
#             print(f'\toutside at {i}, pipe {val}, count {inside_count}, last pipe {last_pipe}')
#             last_pipe = val
#             inside = False
#         else:
#             if (val=='.') and (inside==True):
#                 inside_count+=1
#                 print(f'\tinside at {i}, count {inside_count}')
#         # print(val, inside, inside_count)
#     return inside_count

In [18]:
def count_inside_loop(row):
    side_pipes= {'L','F','7','J','|','S'}
    out_pipes = {'7','J','|','S'}
    in_pipes = {'L','F','|','S'}
    in_corners = {'L','F'}
    out_corners = {'7','J'}
    inside = False
    inside_count = 0
    corner = False
    for i, val in row.items():
        # print(val)
        if (val in in_pipes) and (inside==False):
            inside = True
            print(f'\tinside at {i}, pipe {val}, count {inside_count}')
            if val in in_corners:
                corner = not corner
            if not corner:
                inside = False
        elif (val in out_pipes) and (inside==True):
            inside = False
            print(f'\toutside at {i}, pipe {val}, count {inside_count}')
            if val in out_corners:
                corner = not corner
            if not corner:
                inside = True
            
        else:
            if (val=='.') and (inside==True):
                inside_count+=1
                print(f'\tinside at {i}, count {inside_count}')
        # print(val, inside, inside_count)
    return inside_count

In [19]:
total_inside = 0
for i,row in path_map.iterrows():
    print(f'row {i}')
    total_inside += count_inside_loop(row)
    
total_inside

row 0
row 1
	inside at 1, pipe S, count 0
row 2
	inside at 1, pipe |, count 0
	inside at 2, pipe F, count 0
	outside at 8, pipe 7, count 0
	outside at 9, pipe |, count 0
	inside at 10, count 1
row 3
	inside at 1, pipe |, count 0
	inside at 2, pipe |, count 0
	inside at 8, pipe |, count 0
	inside at 9, pipe |, count 0
row 4
	inside at 1, pipe |, count 0
	inside at 2, pipe |, count 0
	inside at 8, pipe |, count 0
	inside at 9, pipe |, count 0
row 5
	inside at 1, pipe |, count 0
	inside at 2, pipe L, count 0
	outside at 4, pipe 7, count 0
	inside at 5, count 1
	outside at 8, pipe J, count 1
	inside at 9, pipe |, count 1
	inside at 10, count 2
row 6
	inside at 1, pipe |, count 0
	inside at 4, pipe |, count 0
	inside at 6, pipe |, count 0
	inside at 9, pipe |, count 0
row 7
	inside at 1, pipe L, count 0
	outside at 4, pipe J, count 0
	inside at 5, count 1
	outside at 9, pipe J, count 1
row 8


4